# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [55]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df['Age'].fillna(df['Age'].median())
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df['Embarked']

In [63]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
embark_mean = df.groupby(['Embarked'])['Fare'].mean().reset_index()
embark_median = df.groupby(['Embarked'])['Fare'].median().reset_index()
embark_mode = df.groupby(['Embarked'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
embark_max = df.groupby(['Embarked'])['Fare'].max().reset_index()
temp = pd.merge(embark_mean,embark_median, on=['Embarked'], how='left')
temp = pd.merge(temp,embark_mode, on=['Embarked'], how='left')
temp = pd.merge(temp,embark_max, on=['Embarked'], how='left')
temp.columns = ['Embarked','embark_mean','embark_median','embark_mode','embark_max']
df = pd.merge(df, temp, on=['Embarked'], how='left')


In [65]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'embark_mean', 'embark_median', 'embark_mode', 'embark_max']



,Pclass,Age,SibSp,Parch,Fare,embark_mean,embark_median,embark_mode,embark_max
0,3,22.0,1,0,7.2500,27.079812,13.0,8.0500,263.0000
1,1,38.0,1,0,71.2833,59.954144,29.7,7.2292,512.3292
2,3,26.0,0,0,7.9250,27.079812,13.0,8.0500,263.0000
3,1,35.0,1,0,53.1000,27.079812,13.0,8.0500,263.0000
4,3,35.0,0,0,8.0500,27.079812,13.0,8.0500,263.0000


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

<font color=red>結果有改善,但是並不多.只有從0.700提升到0.0703</font>

In [66]:
# 原始特徵 + 邏輯斯迴歸
df_minus = df.drop(['embark_mean', 'embark_median', 'embark_mode', 'embark_max'],axis=1)
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression(solver='lbfgs')
print(f'Score for 原始特徵 + 邏輯斯迴歸 : {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')

Score for 原始特徵 + 邏輯斯迴歸 : 0.7004864906462793


C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [67]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
print(f'Score for 原始特徵 + 邏輯斯迴歸 : {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')

Score for 原始特徵 + 邏輯斯迴歸 : 0.7027589317916567


C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
